# Schritt für Schritt Anleitung
In diesem Notebook zeigen wir Ihnen Schritt für Schritt, wie Sie die Baupublikationen vom Kantonsblatt herunterladen, weiter anreichern und als CSV bzw. Excel-Datei abspeichern können.
Wir beginnen mit dem Laden der notwendigen Bibliotheken sowie dem Setzen einiger globaler Variablen.

## Import libraries and setup logging
Hier importieren wir alle relevanten Bibliotheken, die wir für das Herunterladen, Verarbeiten und Abspeichern der Daten benötigen.

In [37]:
# Bibliotheken für die Datenverarbeitung, Netzwerkzugriff, Geodaten und Fortschrittsanzeigen
import io
import os
import requests
import pandas as pd
import datetime
from tqdm import tqdm
import xml.etree.ElementTree as ET
import zipfile
import geopandas as gpd
from urllib.parse import urlencode


## Global variables
Hier definieren wir ein paar globale Variablen, wie zum Beispiel die Domain für das Kantonsblatt und den Kantons-Code.

In [38]:
# DOMAIN gibt an, von welcher URL die Publikationen geladen werden.
# CANTON legt den Kanton fest, hier 'BS' (Basel-Stadt).
DOMAIN = 'https://www.kantonsblatt.ch'
CANTON = 'BS'

## Setup functions
In den folgenden Zellen definieren wir alle Funktionen, die für das Herunterladen und Aufbereiten der Daten zuständig sind.

### Kantonsblatt
Diese Funktionen kümmern sich um das Laden der Publikationsdaten aus dem Kantonsblatt (API).

In [39]:
# iterate_over_years() ruft iterate_over_pages() für jeden Monat eines gegebenen Jahres auf.
# Dabei wird mit tqdm eine Fortschrittsanzeige für die Jahre dargestellt.
# Ergebnis ist ein DataFrame mit allen Publikationen.
def iterate_over_years():
    start_year = 2019
    df = pd.DataFrame()
    for year in range(start_year, datetime.datetime.now().year + 1):
        print(f'\nGetting data for year {year}\nand month:')
        for month in tqdm(range(1, 13)):
            # Falls wir uns noch im aktuellen Jahr befinden und der Monat in der Zukunft liegt, brechen wir ab.
            if year == datetime.datetime.now().year and month > datetime.datetime.now().month:
                break
            df_month = iterate_over_pages(year, month)
            df = pd.concat([df, df_month])
    return df

In [40]:
# iterate_over_pages(year, month) holt alle Publikationen für einen bestimmten Monat eines bestimmten Jahres.
# Dafür wird eine Pagination über alle Seiten genutzt. Die CSV-Daten werden in einen DataFrame geladen.
# Anschließend wird add_columns() auf die Daten angewendet.
def iterate_over_pages(year, month):
    base_url = f'{DOMAIN}/api/v1/publications/csv?publicationStates=PUBLISHED&cantons={CANTON}'
    start_date = f'&publicationDate.start={year}-{month}-01'
    end_date = f'&publicationDate.end={year}-{month + 1}-01' if month < 12 else f'&publicationDate.end={year + 1}-01-01'
    url = f'{base_url}{start_date}{end_date}'

    page = 0
    next_page = f'{url}&pageRequest.page={page}'
    df = pd.DataFrame()

    while True:
        r = requests.get(next_page)
        r.raise_for_status()
        df_curr_page = pd.read_csv(io.StringIO(r.content.decode('utf-8')), sep=';')
        if df_curr_page.empty:
            # Wenn keine Daten mehr geladen werden können, brechen wir die Schleife ab.
            break

        # Füge zusätzliche Spalten (z.B. URLs) hinzu
        df_curr_page = add_columns(df_curr_page)

        df = pd.concat([df, df_curr_page])
        page = page + 1
        next_page = f'{url}&pageRequest.page={page}'

    return df

In [41]:
# add_columns() generiert zusätzliche Spalten für den direkten Zugriff auf PDF, XML und die Web-Ansicht
# basierend auf der ID einer Publikation.
def add_columns(df):
    df['url_kantonsblatt'] = df['id'].apply(lambda x: f'{DOMAIN}/#!/search/publications/detail/{x}')
    df['url_pdf'] = df['id'].apply(lambda x: f'{DOMAIN}/api/v1/publications/{x}/pdf')
    df['url_xml'] = df['id'].apply(lambda x: f'{DOMAIN}/api/v1/publications/{x}/xml')
    return df

In [42]:
# get_rubric_from_api() lädt Rubriken (Ober- und Unterrubriken) aus der Kantonsblatt-API
# und konvertiert sie in DataFrames.
def get_rubric_from_api():
    url = f'{DOMAIN}/api/v1/rubrics'
    r = requests.get(url)
    r.raise_for_status()
    df = pd.read_json(io.StringIO(r.content.decode('utf-8')))

    # Die Spalte 'code' wird in 'rubric' umbenannt
    df = df.rename(columns={'code': 'rubric'})

    # Normalisierung der Struktur, um Rubriken und deren Namen in verschiedenen Sprachen zu erhalten
    df = pd.concat([df[['rubric', 'subRubrics']], pd.json_normalize(df['name'])], axis=1)
    df_rubric = df.rename(columns={'en': 'rubric_en', 'de': 'rubric_de', 'fr': 'rubric_fr',
                                   'it': 'rubric_it'})[['rubric', 'rubric_en', 'rubric_de', 'rubric_fr', 'rubric_it']]

    # Explodiere die Unterrubriken und normalisiere diese ebenfalls
    df = df.explode('subRubrics').reset_index(drop=True)
    df = pd.json_normalize(df['subRubrics'])[['code', 'name.en', 'name.de', 'name.fr', 'name.it']]
    df_subRubric = df.rename(columns={'code': 'subRubric', 'name.en': 'subRubric_en', 'name.de': 'subRubric_de',
                                      'name.fr': 'subRubric_fr', 'name.it': 'subRubric_it'})
    return df_rubric, df_subRubric

In [43]:
# get_tenants_from_api() lädt die verfügbaren 'tenantCodes' (z.B. Kantons-Organisationseinheiten)
# und liefert ein Dictionary zur Umwandlung von Code zu Titel.
def get_tenants_from_api():
    url = f'{DOMAIN}/api/v1/tenants'
    r = requests.get(url)
    r.raise_for_status()
    tenants = r.json()
    return {tenant['id']: tenant['title']['de'] for tenant in tenants}

### Baupublikationen
Die folgenden Funktionen helfen dabei, die Daten einer Baupublikation zu extrahieren, z.B. aus XML.


In [44]:
# add_content_to_row(row) lädt für eine bestimmte Baupublikation das zugehörige XML und
# konvertiert die Inhalte in ein DataFrame. Dadurch erhalten wir eine fein aufgeschlüsselte Struktur.
def add_content_to_row(row):
    content, _ = get_content_from_xml(row['url_xml'])
    df_content = xml_to_dataframe(content)

    # Speichere den reinen XML-Inhalt (als Byte-String) in der Spalte 'content'
    row['content'] = ET.tostring(content, encoding='utf-8')

    # Übertrage alle Spalten aus dem ursprünglichen row in das neue df_content
    for col in row.index:
        if col in df_content.columns:
            # Falls Spalte schon existiert, kombiniere vorhandene Werte
            df_content[col] = df_content[col].combine_first(pd.Series([row[col]] * len(df_content)))
        else:
            # Erstelle eine neue Spalte mit dem Wert aus 'row'
            df_content[col] = pd.Series([row[col]] * len(df_content))

    return df_content

In [45]:
# get_content_from_xml(url) lädt das XML für eine Publikation herunter und gibt
# den XML-Knoten <content> und <attachments> zurück.
def get_content_from_xml(url):
    r = requests.get(url)
    r.raise_for_status()
    xml_content = r.text
    root = ET.fromstring(xml_content)
    content = root.find('content')
    attachments = root.find('attachments')
    return content, attachments

In [46]:
# xml_to_dataframe(root) durchläuft das XML und erstellt ein DataFrame,
# wobei jeder Pfad einen Spaltennamen ergibt und der Text zugeordnet wird.
def xml_to_dataframe(root):
    def traverse(node, path='', path_dict=None):
        if path_dict is None:
            path_dict = {}

        # Wenn der Knoten weitere Kinder hat, durchlaufe sie rekursiv
        if list(node):  # If the node has children
            for child in node:
                child_path = f'{path}_{child.tag}' if path else child.tag
                traverse(child, child_path, path_dict)
        else:  # Falls Blattknoten, nimm den Text
            value = node.text.strip() if node.text and node.text.strip() else ''
            if path in path_dict:
                path_dict[path].append(value)
            else:
                path_dict[path] = [value]
        return path_dict

    path_dict = traverse(root)

    # Finde die maximale Länge der Listen, um das DataFrame gleichmäßig aufzufüllen
    max_len = max(len(v) for v in path_dict.values())

    # Passe die Listen in path_dict an die maximale Länge an
    expanded_data = {
        k: (
            v * max_len if len(v) == 1 
            else v + [''] * (max_len - len(v))
        )
        for k, v in path_dict.items()
    }

    df = pd.DataFrame(expanded_data)
    return df

In [47]:
# legal_form_code_to_name(df) wandelt Firmenrechtsformen (Codes) in ihre Klartexte um (z.B. AG, GmbH, etc.).
# Dazu wird die i14y-API angefragt.
def legal_form_code_to_name(df):
    url_i14y = "https://input.i14y.admin.ch/api/ConceptInput/08dad8ff-f18a-560b-bfa6-20767f2afb17/codelistEntries?page=1&pageSize=10000"
    response = requests.get(url_i14y)
    response.raise_for_status()
    legal_forms = response.json()

    # Mapping von Code -> deutscher Name
    code_to_german_name = {entry['value']: entry['name']['de'] for entry in legal_forms}

    # Ersetze Codes in den relevanten Spalten
    df['projectFramer_company_legalForm'] = df['projectFramer_company_legalForm'].map(code_to_german_name)
    df['buildingContractor_company_legalForm'] = df['buildingContractor_company_legalForm'].map(code_to_german_name)
    return df

### Parzellen in BS für Baupublikationen
Die folgenden Funktionen dienen dazu, die Grundstücks-Geometrien (Parzellen) aus den OpenData des Kantons Basel-Stadt zu laden und zu den Baupublikationen hinzuzufügen.

In [48]:
# get_geometry(gdf, row) sucht zu einer gegebenen Parzellennummer (und Sektionsangabe) die
# passende Geometrie in gdf (ein GeoDataFrame) und gibt diese zurück.
def get_geometry(gdf, row):
    parzellennummer = row['districtCadastre_relation_plot']
    section = row['districtCadastre_relation_section']
    numbers = parzellennummer.split(',')
    filtered_gdf = gdf[(gdf['parzellennu'].isin(numbers)) & (gdf['r1_sektion'] == section)]
    geometries = filtered_gdf['geometry']
    if len(geometries) == 1:
        # Falls genau eine Geometrie vorliegt
        return geometries.iloc[0]
    else:
        # Falls mehrere Parzellen zusammengefasst werden sollen
        return geometries.union_all()

In [49]:
# get_parzellen(df) lädt die Parzellen (Shape-Dateien) von data.bs.ch, entpackt sie
# und reichert das übergebene DataFrame mit den entsprechenden Geometrien an.
def get_parzellen(df):
    # Download des Shapefile-Archivs
    url_parzellen = 'https://data.bs.ch/explore/dataset/100201/download/?format=shp'
    r = requests.get(url_parzellen)

    # ZIP entpacken
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall('100201')

    # Shapefile einlesen
    path_to_shp = os.path.join('100201', '100201.shp')
    gdf = gpd.read_file(path_to_shp, encoding='utf-8')

    # Fehlende Werte in 'districtCadastre_relation_plot' auffüllen
    df.loc[df['districtCadastre_relation_plot'].isna(), 'districtCadastre_relation_plot'] = ''
    df['districtCadastre_relation_plot'] = df['districtCadastre_relation_plot'].astype(str)

    # Parzellennummern ggf. mit führenden Nullen auffüllen
    df['districtCadastre_relation_plot'] = df['districtCadastre_relation_plot'].apply(correct_parzellennummer)

    # Hole zu jeder Zeile im DF die passende Geometrie
    df['geometry'] = df.apply(lambda x: get_geometry(gdf, x), axis=1)

    # Erstelle einen Link zur entsprechenden Parzelle auf data.bs.ch
    df['url_parzellen'] = df.apply(
        lambda row: 'https://data.bs.ch/explore/dataset/100201/table/?' +
                    urlencode({
                        'refine.r1_sektion': row['districtCadastre_relation_section'],
                        'q': 'parzellennummer: ' + ' OR '.join(
                            row['districtCadastre_relation_plot'].split(',')
                        )}), axis=1)

    return df

In [50]:
# correct_parzellennummer(parzellennummer) macht aus z.B. '48' -> '0048'.
# Damit passt es zum Format der Shapefile-Daten.
def correct_parzellennummer(parzellennummer):
    parts = parzellennummer.split(',')
    parts = [part.strip() for part in parts]
    corrected = [num.zfill(4) for num in parts]
    return ','.join(corrected)

## Get Kantonsblatt data
Hier werden alle Jahre ab 2019 bis zum aktuellen Datum durchlaufen und sämtliche Publikationen abgerufen.
**Achtung**: Dieses Skript lädt potenziell sehr viele Daten und kann entsprechend lange laufen!

In [ ]:
# Dieser Aufruf kann viel Zeit in Anspruch nehmen, da er alle Monate ab 2019 durchgeht.
df_kantonsblatt = iterate_over_years()

### Rubriken hinzufügen
Hier werden die Codes für Rubriken und SubRubriken in Klartext umgewandelt.

In [ ]:
# Lade die Rubriken und führe sie mit den Publikationen zusammen.
df_rubric, df_subRubric = get_rubric_from_api()
df_kantonsblatt = df_kantonsblatt.merge(df_rubric, how='left', on='rubric')
df_kantonsblatt = df_kantonsblatt.merge(df_subRubric, how='left', on='subRubric')

### Tenants hinzufügen
Hier werden die Tenant-Codes (Organisationseinheiten) in sprechende Namen umgewandelt.

In [ ]:
# Mappen der 'primaryTenantCode' und 'secondaryTenantsTenantCode' auf deren Namen.
tenant_code_to_name = get_tenants_from_api()
df_kantonsblatt['primaryTenantName'] = df_kantonsblatt['primaryTenantCode'].map(tenant_code_to_name)
df_kantonsblatt['secondaryTenantsTenantName'] = df_kantonsblatt.loc[df_kantonsblatt['secondaryTenantsTenantCode'].notna(), 'secondaryTenantsTenantCode'].str.split(',').apply(lambda x: ','.join([tenant_code_to_name.get(y) for y in x]))

In [ ]:
# Speichere die kombinierten Kantonsblatt-Daten in CSV/XLSX.
df_kantonsblatt.to_csv('kantonsblatt.csv', index=False)
df_kantonsblatt.to_excel('kantonsblatt.xlsx', index=False)

## Get Baupublikationen
Aus den Kantonsblatt-Daten filtern wir hier explizit die Baupublikationen heraus (z.B. SubRubric == 'BP-BS10').

In [23]:
# Wähle nur die Zeilen, die SubRubric == 'BP-BS10' haben.
df_baupublikationen = df_kantonsblatt[df_kantonsblatt['subRubric'] == 'BP-BS10'][['id', 'url_xml']]

In [28]:
# Für jede Baupublikation wird das XML geladen und daraus Detailinformationen extrahiert.
all_data = []
# Print number of rows
print(f'Number of rows: {len(df_baupublikationen)}it to be done')
for index, row in tqdm(df_baupublikationen.iterrows()):
    df_content = add_content_to_row(row)
    all_data.append(df_content)

# Sammle alle Einträge in einem DataFrame
df_baupublikationen = pd.concat(all_data, ignore_index=True)  # Concatenate all dataframes

Number of rows: 182it to be done


182it [01:37,  1.87it/s]


In [31]:
# Spaltennamen vereinfachen, um z.B. lange Namen zu kürzen.
df_baupublikationen.columns = df_baupublikationen.columns.str.replace('_legalEntity_multi_companies_', '_')
df_baupublikationen.columns = df_baupublikationen.columns.str.replace('_multi_companies_', '_')

In [32]:
# Wandelt Rechtsform-Codes (AG, GmbH, etc.) in Klartexte um
df_baupublikationen = legal_form_code_to_name(df_baupublikationen)

### Last but not least: Get Parzellen
Zum Schluss fügen wir die Parzellengeometrien hinzu, damit wir wissen, auf welchen Grundstücken gebaut wird.
Das Laden der Shapefile-Daten kann einen Moment dauern.

In [35]:
# Füge die Geometrie (Shapefile) auf Basis der Parzellennummern hinzu.
df_baupublikationen = get_parzellen(df_baupublikationen)

<ipython-input-17-fbec35bea8fb>:11: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  return geometries.unary_union
<ipython-input-17-fbec35bea8fb>:11: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  return geometries.unary_union
<ipython-input-17-fbec35bea8fb>:11: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  return geometries.unary_union
<ipython-input-17-fbec35bea8fb>:11: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  return geometries.unary_union
<ipython-input-17-fbec35bea8fb>:11: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  return geometries.unary_union
<ipython-input-17-fbec35bea8fb>:11: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  return geometrie

In [36]:
# Speichere die Baupublikationen inkl. Parzellen-Geometrie
df_baupublikationen.to_csv('baupublikationen.csv', index=False)
df_baupublikationen.to_excel('baupublikationen.xlsx', index=False)